In [2]:
import numpy as np
import scipy.signal as sf
import soundfile as sf
import librosa
import librosa.display
import IPython.display 
from scipy.io import wavfile
import matplotlib.pyplot as plt


In [2]:
def crosstalk_cancel(left, right, ir_30, ir_330):

    
    """ Crosstalk cancellation to an audio file
    
    Parameters
    ----------
    left  : audiofile left ear
    
    right : audiofile right ear
    
    ir_30 : impulse response ipsilateral ear
    
    ir_330: impulse response contralateral ear
    
    Returns
    -------
    stereo file : stereo[0,:]=left, stereo[1,:]=right

    """
    
    h_30 = np.concatenate((np.zeros(512), ir_30[0,:]))
    h_330 = np.concatenate((np.zeros(512), ir_30[1,:]))

    nFFT = 2**14
    H_30 = np.fft.fft(h_30, nFFT)
    H_330 = np.fft.fft(h_330, nFFT)

    A = - H_330 / H_30
    B = 1 / (1 - ((H_330/H_30)**2.0))
    C = 1 / H_30

    a = np.real(np.fft.ifft(A))
    b = np.real(np.fft.ifft(B))
    c = np.real(np.fft.ifft(C))

    a = np.roll(a, 3000)
    b = np.roll(b, 3050)
    c = np.roll(c, 1850)



    left_speaker = left + np.convolve(right, a, 'same')
    left_speaker = np.convolve(left_speaker, b)
    left_speaker = np.convolve(left_speaker, c)

    right_speaker = right + np.convolve(left, a, 'same')
    right_speaker = np.convolve(right_speaker, b)
    right_speaker = np.convolve(right_speaker, c)


    stereo = np.vstack([left_speaker,right_speaker])
    stereo = np.transpose(stereo)
    
    return stereo

 
    


In [6]:
""" Running function and exporting wav file  """


y, sr = librosa.load('Audio/Haircut16-44p1.aif', sr=44100, mono=False)
left = y[0]
right = y[1]



ir_30, sr_30 = librosa.load('Audio/azi_30,0_ele_0,0.wav', sr=44100, mono=False)
ir_330, sr_330 = librosa.load('Audio/azi_330,0_ele_0,0.wav', sr=44100, mono=False)


stereo = crosstalk_cancel(left, right, ir_30, ir_330)


wavfile.write("haircut_xtc.wav", 44100, stereo)
    
    